## Future forecasting:

In [77]:
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from xgboost import plot_importance
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import numpy as np

In [78]:
## PULL Functions from custom functions
from custom_functions import bertopic_addons as cfc

## you will need to change the name below
currenttable = 'topified_vectorized_Science1900_2023'
aws_dfs = cfc.pull_aws_sql_database(currenttable)

Step 1/7: Loading the dataset ...


In [ ]:
## build in functionality for time frame no.of.months

In [79]:
aws_dfs.columns

Index(['paperId', 'externalIds', 'publicationVenue', 'title', 'abstract',
       'year', 'referenceCount', 'citationCount', 'influentialCitationCount',
       'fieldsOfStudy', 's2FieldsOfStudy', 'publicationTypes',
       'publicationDate', 'title_abstract', 'topic_code', 'topic_list',
       'x_vector', 'y_vector', 'z_vector'],
      dtype='object')

In [192]:

moving_average_years = 1
dataframe = aws_dfs
dataframe['count'] = 1
# Group by "Topic" and "Timestamp" and aggregate "Frequency"
## metric can be 'count', 'citationCount', 'influentialCitationCount'
type_of_metric = 'citationCount'
timeframe = 'Y'

In [143]:
aws1_dfs.shape

(53833, 20)

In [141]:
aws1_dfs[aws1_dfs['publicationDate'] != 'None'].shape

(46284, 20)

In [193]:
## you can change this to all topics 
aws1_dfs= dataframe
aws1_dfs['topic_code'] = pd.to_numeric(aws1_dfs['topic_code'], downcast='integer')
aws1_dfs[type_of_metric] = pd.to_numeric(aws1_dfs[type_of_metric], downcast='integer')
aws1_dfs = aws1_dfs[aws1_dfs['publicationDate'] != 'None']
aws1_dfs['publicationDate'] = pd.to_datetime(aws1_dfs['publicationDate'], format='%Y-%m-%d')
aws1_dfs['date'] = aws1_dfs['publicationDate'].dt.to_period(timeframe)
aws1_dfs['date']=aws1_dfs['date'].astype(str)
aws1_dfs['date']=pd.to_datetime(aws1_dfs['date'])
# ==== LEGACY NO NEED :)) aws1_dfs= aws1_dfs[aws1_dfs['topic_code']!=-1]

In [195]:
def plot_bertopic_data_into_future(dataframe, topic_number, future_years, moving_average, type_of_metric, timeframe):
    
    
    
    
    ##### ============================ CLEANING UP DATAFRAME 
    ## number of years into the future you can forecast
    if timeframe == 'M':
        n = future_years*12
        moving_average = moving_average*12
    else: 
        n = future_years
        
    ## you can change this to all topics 
    aws1_dfs= dataframe
    aws1_dfs['topic_code'] = pd.to_numeric(aws1_dfs['topic_code'], downcast='integer')
    aws1_dfs[type_of_metric] = pd.to_numeric(aws1_dfs[type_of_metric], downcast='integer')
    aws1_dfs = aws1_dfs[aws1_dfs['publicationDate'] != 'None'].reset_index()
    aws1_dfs['publicationDate'] = pd.to_datetime(aws1_dfs['publicationDate'], format='%Y-%m-%d')
    aws1_dfs['date'] = aws1_dfs['publicationDate'].dt.to_period(timeframe)
    aws1_dfs['date']=aws1_dfs['date'].astype(str)
    aws1_dfs['date']=pd.to_datetime(aws1_dfs['date'])
    # ==== LEGACY NO NEED :)) aws1_dfs= aws1_dfs[aws1_dfs['topic_code']!=-1]

    ## code to change the date time, for the moment use year
    grouped_df = aws1_dfs.groupby(['topic_code', 'date'])[type_of_metric].sum().reset_index()

    pivot_df = grouped_df.pivot(index='date', columns='topic_code', values=type_of_metric)

    pivot_df.fillna(0, inplace=True)
    pivot_df = pivot_df.reset_index()
        
    # Define the target column (Topic you want to forecast)
    target_topic = topic_number  # Change this to the Topic you want to forecast
        
    # convert target topic to a moving average
    ## now predicting the moving average
    pivot_df[target_topic] = pivot_df[target_topic].rolling(moving_average_years).mean()


    # Dropping last n rows using drop
    target_column = pivot_df[target_topic]

    # forget about the lost values
    ## dropped the oldest columns
    target_column.drop(target_column.head(n).index, inplace = True)

    ## pivot_df['year'] + pd.offsets.DateOffset(years=5)
    untarget_columns = pivot_df.drop(target_topic, axis=1)

    ## keep this for later -- these are your forecasting columns 
    X_forecasting_data = untarget_columns.tail(n)
    X_forecasting_years = untarget_columns['date'].tail(n) + pd.offsets.DateOffset(years=future_years)
    X_forecasting_data['date'] = X_forecasting_years
    X_forecasting = X_forecasting_data.drop(columns=['date'])

    ##  drop the columns you keep for forecasting
    untarget_columns.drop(target_column.tail(n).index, inplace = True)

    df = pd.concat([untarget_columns, target_column.reset_index(drop=True)], axis=1)

    ## shift the prediction of each year
    df['date'] = df['date'] + pd.offsets.DateOffset(years=future_years)

    pivot_df = df    
    
    
    
    
    
    ### ================== TEST TRAIN SPLIT 
    
    
    
    
    
    # Separating the training set and testing set
    train_data=pivot_df[pivot_df['date'].dt.year<2011].reset_index(drop = True)
    test_data=pivot_df[pivot_df['date'].dt.year>2010].reset_index(drop = True)

    # Prepare the training and testing data
    X_train = train_data.drop(target_topic, axis=1)
    X_test = test_data.drop(target_topic, axis=1)

    # Shift the target column to align with next year's frequency
    y_train = train_data[target_topic].shift(-1).dropna()
    y_test = test_data[target_topic].shift(-1).dropna()

    # Exclude the 'YearMonth' column from the training and testing data
    X_train = train_data.drop(columns=['date', target_topic]).iloc[:-1]
    X_test = test_data.drop(columns=['date', target_topic]).iloc[:-1]
    
    
    
    ## ======================= MODEL 
    
    
    

    # Create an XGBoost regressor
    model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, max_depth=3)

    # Fit the model on the training data
    model.fit(X_train, y_train)

    # Make predictions on the testing data
    y_pred = model.predict(X_test)
    y_forecasting = model.predict(X_forecasting)

    # Create a DataFrame with YearMonth and the predicted values
    y_pred_df = pd.DataFrame({'date': test_data['date'].iloc[:-1], 'Predicted': y_pred})
    y_fore_df = pd.DataFrame({'date': X_forecasting_years, 'Forecasted': y_forecasting})

    # Merge the predicted DataFrame with the original test_data DataFrame
    merged_data = pd.merge(y_pred_df, test_data, on='date')
    merged_forecasted_data = pd.merge(y_fore_df, X_forecasting_data, on='date')

    #print(merged_forecasted_data)
    # Calculate the root mean squared error (RMSE)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    #print(f"Root Mean Squared Error (RMSE): {rmse}")
    
    
    
    ## =============================   BUILDING DATA TABLE 
    
    
    
    y_actual_test_df = test_data[['date',target_topic]]
    y_actual_train_df = train_data[['date',target_topic]]
    y_fore_df =y_fore_df.rename(columns={'Forecasted': target_topic})
    y_pred_df =y_pred_df.rename(columns={'Predicted': target_topic})

    y_actual_test_df['value'] = 'Actual_Test'
    y_actual_train_df['value'] = 'Actual_Train'
    y_fore_df['value'] = 'Forecasted'
    y_pred_df['value'] = 'Predicted_Test'

    final_data = pd.concat([y_actual_test_df,y_actual_train_df,y_fore_df,y_pred_df], axis=0)
    final_data =final_data.rename(columns={target_topic: 'quantity'})
    final_data['topic_code'] = target_topic
    final_data['RMSE'] = rmse
    final_data['metric'] = type_of_metric
    
    
    
    """# Plot the predicted vs. actual values along with y_train
    plt.figure(figsize=(12, 6))
    plt.plot(merged_data['year'], merged_data['Predicted'], label='Predicted', marker='x')
    plt.plot(merged_data['year'], merged_data[target_topic], label='Actual', marker='o')
    plt.plot(train_data['year'].iloc[:-1], y_train, label='Train', linestyle='--', color='gray')
    plt.plot(merged_forecasted_data['year'], merged_forecasted_data['Forecasted'], label='Forecasted', marker='+')
    
    plt.xlabel('Time')
    plt.ylabel(type_of_metric)
    plt.title(f'Topic {target_topic} Citation Forecast (RMSE: {rmse:.2f})')
    plt.legend()
    plt.grid(True)
    # Feature importance plot (optional)
    plot_importance(model)
    plt.show()"""
    
    return final_data
    

In [196]:
## create an ordered list of topic codes 
topic_code_list = list(pd.Series(dataframe['topic_code'].sort_values()).unique())

## create a dataframe to concat later 
growing_df = pd.DataFrame()

## for loop through the topic code list 
for j in ['count', 'citationCount', 'influentialCitationCount']:
    for i in topic_code_list:
        current_df = plot_bertopic_data_into_future(dataframe, i, 5, 4, j, timeframe)
        growing_df = pd.concat([current_df, growing_df]) 
    
## reset the index and drop the inxiex column 
growing_df = growing_df.reset_index().drop('index',axis=1)

## match the topic code to the topic description 
topic_codes = dataframe[['topic_code','topic_list']].drop_duplicates().reset_index().drop('index',axis=1)

## merge the topics descriptions to the data set
full_datase= growing_df.merge(topic_codes, on='topic_code')

In [190]:
from custom_functions import bertopic_addons as cfc

In [197]:
name_for_dataframe = 'forecasted_predicted_actual_Science_1900_2023'
cfc.dataframe_to_aws_sql(full_datase,name_for_dataframe)
cfc.pull_aws_sql_database(name_for_dataframe)

Step 1/7: Loading the dataset ...


,date,quantity,value,topic_code,RMSE,metric,topic_list
0,2011-01-01,81.000000,Actual_Test,117,26.053664,influentialCitationCount,117_multiple sclerosis_sclerosis_multiple_axonal
1,2012-01-01,18.000000,Actual_Test,117,26.053664,influentialCitationCount,117_multiple sclerosis_sclerosis_multiple_axonal
2,2013-01-01,5.000000,Actual_Test,117,26.053664,influentialCitationCount,117_multiple sclerosis_sclerosis_multiple_axonal
3,2014-01-01,2.000000,Actual_Test,117,26.053664,influentialCitationCount,117_multiple sclerosis_sclerosis_multiple_axonal
4,2015-01-01,0.000000,Actual_Test,117,26.053664,influentialCitationCount,117_multiple sclerosis_sclerosis_multiple_axonal
...,...,...,...,...,...,...,...
48547,2018-01-01,167.447266,Predicted_Test,-1,31.413152,count,-1_patients_study_results_disease
48548,2019-01-01,169.941528,Predicted_Test,-1,31.413152,count,-1_patients_study_results_disease
48549,2020-01-01,173.979416,Predicted_Test,-1,31.413152,count,-1_patients_study_results_disease
48550,2021-01-01,177.606995,Predicted_Test,-1,31.413152,count,-1_patients_study_results_disease


In [73]:
working_data = cfc.pull_aws_sql_database(name_for_dataframe)

Step 1/7: Loading the dataset ...


In [76]:
working_data[['topic_code', 'year', 'metric']]

,topic_code,year,metric
0,117,2011-01-01,influentialCitationCount
1,117,2012-01-01,influentialCitationCount
2,117,2013-01-01,influentialCitationCount
3,117,2014-01-01,influentialCitationCount
4,117,2015-01-01,influentialCitationCount
...,...,...,...
48547,-1,2018-01-01,count
48548,-1,2019-01-01,count
48549,-1,2020-01-01,count
48550,-1,2021-01-01,count
